# import

In [26]:
import pandas as pd

# 보통 현재 작업 폴더(CWD)가 src 이므로 부모(프로젝트 루트)를 sys.path에 추가
import sys, os
from pathlib import Path

ROOT = Path.cwd().parent
if (ROOT / "utils").exists():
    sys.path.insert(0, str(ROOT))
else:
    # 혹시 구조가 더 깊을 때 대비
    ROOT2 = ROOT.parent
    if (ROOT2 / "utils").exists():
        sys.path.insert(0, str(ROOT2))

from utils.angular_util import compute_joint_angles, compute_connected_unit_vectors, flatten_vectors, compute_face_hand_vectors

# 라벨 데이터

In [27]:
sign_code_df = pd.read_csv("../../data/sign_code.csv")

In [28]:
sign_code_df

,label,sign_num,seq_Id,sign_text,seq_count
0,0,1,0,1,1
1,1,2,0,2,1
2,2,3,0,3,1
3,3,4,0,4,1
4,4,5,0,5,1
5,5,6,0,6,1
6,6,7,0,7,1
7,7,8,0,8,1
8,8,9,0,9,1
9,9,10,0,10,2


# 데이터 병합

## 수집한 데이터 합치기

In [29]:
HAND_COUNT = 21 * 3
POSE_COUNT = 11 * 3

## 헤더 추가
🚨 주의!!!! 이미 헤더가 있는 경우 또 추가될 수 있음!

In [30]:
# for i in range(7, 8):
#     data = pd.read_csv(f"../../data/sign_data/sign_data_{i}.csv")
    # data.insert(0, 'label', [i] * len(data))
    # columns = [i for i in range(len(data.columns) - 1)]
    # columns.insert(0, 'label')
    # data.columns = columns
    # data.to_csv(f"../../data/sign_data/sign_data_{i}.csv", index=False)

In [31]:
# i = 5
# data = pd.read_csv(f"../../data/sign_data/sign_data_{i}.csv", index_col=False)
# data.loc[:, 'label'] = i
# data.to_csv(f"../../data/sign_data/sign_data_{i}.csv", index=False)

## 병합

In [32]:
merged_data = pd.DataFrame()

for i, label in enumerate([0, 1, 2, 3, 4, 5, 6, 14, 15, 16, 24]):
    print(label)
    data = pd.read_csv(f"../../data/sign_data/sign_data_{label}.csv", index_col=False)
    data = data.drop(['label'], axis=1)
    data.loc[:, 'label'] = i
    data = data.sample(n = 500, replace=False)
    merged_data = pd.concat([merged_data, data], ignore_index=True)

df = merged_data

0
1
2
3
4
5
6
14
15
16
24


In [33]:
df

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,label
0,0,0,0,0,0,0,0,0,0,0,...,0.414361,0.440044,-0.354263,0.520832,0.508829,-0.611334,0.465103,0.511247,-0.613954,0
1,0,0,0,0,0,0,0,0,0,0,...,0.412517,0.437244,-0.415101,0.504196,0.508664,-0.675661,0.455466,0.505210,-0.673760,0
2,0,0,0,0,0,0,0,0,0,0,...,0.414744,0.451149,-0.462809,0.516315,0.522498,-0.709658,0.463946,0.521893,-0.714134,0
3,0,0,0,0,0,0,0,0,0,0,...,0.411313,0.475024,-0.272519,0.510712,0.556643,-0.479596,0.453017,0.559444,-0.478271,0
4,0,0,0,0,0,0,0,0,0,0,...,0.373081,0.438745,-0.361331,0.477069,0.519971,-0.663967,0.412254,0.516172,-0.646902,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5495,0,0,0,0,0,0,0,0,0,0,...,0.391347,0.483148,-0.234045,0.491744,0.557215,-0.501289,0.433827,0.560654,-0.483326,10
5496,0,0,0,0,0,0,0,0,0,0,...,0.391857,0.490386,-0.267387,0.495247,0.574444,-0.531948,0.434930,0.573819,-0.512500,10
5497,0,0,0,0,0,0,0,0,0,0,...,0.384573,0.404313,-0.380825,0.489548,0.497569,-0.662262,0.427853,0.500207,-0.659424,10
5498,0,0,0,0,0,0,0,0,0,0,...,0.364703,0.437101,-0.350830,0.484409,0.540456,-0.612699,0.415996,0.545249,-0.603866,10


In [34]:
# merged_data.to_csv("../../data/merge_num_7_data.csv", index=False)

# 데이터 불러오기

In [35]:
# import pandas as pd
# df = pd.read_csv("../../data/merge_num_7_data.csv")

## 클래스 확인

In [36]:
df['label'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

## x, y 데이터

In [37]:
y = df['label']
x = df.drop(['label'], axis=1)

print(x.shape, y.shape)

(5500, 159) (5500,)


# 데이터 전처리

In [38]:
right_points_x = x.iloc[:, 63:63+63]
face_points_x = x.iloc[:, 63+63:]
nose_point_x = face_points_x.iloc[:, 0]
print(right_points_x.shape, face_points_x.shape, nose_point_x.shape)

(5500, 63) (5500, 33) (5500,)


In [39]:
angles_list = []
vector_list = []
face_hand_vector_list = []
for i in range(len(right_points_x)):
    hand_row = right_points_x.iloc[i].tolist()
    face_row = face_points_x.iloc[i].tolist()
    angles, _, _ = compute_joint_angles(hand_row)
    angles_list.append(angles)

    _, _, vector = compute_connected_unit_vectors(hand_row, return_flat=True)
    vector_list.append(vector)

    _, _, face_hand_vector = compute_face_hand_vectors(face_row, hand_row)
    face_hand_vector_list.append(face_hand_vector)

print(len(angles_list[0]), len(vector_list[0]))

angular_x = pd.DataFrame(angles_list, columns=None)
vector_x = pd.DataFrame(vector_list, columns=None)
face_hand_vector_x = pd.DataFrame(face_hand_vector_list, columns=None)

print(angular_x.shape, vector_x.shape, face_hand_vector_x.shape)

15 72
(5500, 15) (5500, 72) (5500, 54)


In [40]:
result_x = pd.concat([angular_x, vector_x, face_hand_vector_x], axis=1)
result_x.columns = range(len(result_x.columns))

print(result_x.shape)

(5500, 141)


In [41]:
# result_x.to_csv("../../data/angle_vector_num_7_data.csv", index=False)

## 데이터 분할하기

In [42]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(result_x, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

## 데이터 스케일링

In [43]:
# from sklearn.preprocessing import MinMaxScaler
# mms = MinMaxScaler()
# x_train = mms.fit_transform(x_train)
# x_test = mms.transform(x_test)

## 모델 학습

In [44]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np

model = XGBClassifier(n_estimators=200, max_depth=3, learning_rate=0.01)
#model = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=42)
#model = LogisticRegression(solver='liblinear', max_iter=100)
model.fit(x_train, y_train)

score = model.score(x_test, y_test)

print(f"Accuracy: {np.round(score*100, 2)}%")

Accuracy: 99.09%


In [45]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.98      0.98       100
           1       0.94      0.99      0.97       100
           2       0.99      0.95      0.97       100
           3       0.99      1.00      1.00       100
           4       1.00      0.99      0.99       100
           5       0.99      1.00      1.00       100
           6       1.00      0.99      0.99       100
           7       1.00      1.00      1.00       100
           8       1.00      1.00      1.00       100
           9       1.00      1.00      1.00       100
          10       1.00      1.00      1.00       100

    accuracy                           0.99      1100
   macro avg       0.99      0.99      0.99      1100
weighted avg       0.99      0.99      0.99      1100



## 모델 저장하기

In [47]:
import joblib

joblib.dump(model, "../../models/xgb_sample_model.pkl")

['../../models/xgb_sample_model.pkl']